# 1000維 10000筆資料跑統計檢定

1000維常態分佈 一萬筆資料跑檢定 因為最後要看nmist 要多維，看統計檢定可不可以過  
=> 或是直接下載mnist資料分兩半看母體是不是一樣的


兩個方案：
* A) 人工產生的 1000 維常態、1 萬筆
* B) 直接用 MNIST 分兩半比「是否同母體」。

而上述方案都包含：
* **Energy distance(置換檢定)**
* **MMD(核兩樣本)**
* **Hotelling’s （用縮減共變數避免高維不穩）**
* 再加**逐維 t-test + FDR與PCA 視覺化**。

---

## Setting

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, f
from statsmodels.stats.multitest import multipletests
from sklearn.decomposition import PCA
from sklearn.covariance import LedoitWolf
from sklearn.metrics.pairwise import rbf_kernel

rng = np.random.default_rng(0)

def energy_stat(X, Y):
    # 論文定義的兩樣本 energy distance 的「統計量」(未標準化常數對p-value不影響)
    # E||X - Y|| - 0.5 E||X - X'|| - 0.5 E||Y - Y'||
    def _avg_pairwise_norm(A, B):
        # 平均 L2 距離
        a2 = np.sum(A*A, axis=1)[:,None]
        b2 = np.sum(B*B, axis=1)[None,:]
        D2 = a2 + b2 - 2*A@B.T
        np.maximum(D2, 0, out=D2)
        return np.mean(np.sqrt(D2))
    return _avg_pairwise_norm(X, Y) - 0.5*_avg_pairwise_norm(X, X) - 0.5*_avg_pairwise_norm(Y, Y)

def permutation_test(stat_fn, X, Y, n_perm=500, tail='right', subsample=None):
    # 可選擇下採樣加速
    if subsample is not None:
        nX = min(len(X), subsample//2)
        nY = min(len(Y), subsample - nX)
        X = X[rng.choice(len(X), nX, replace=False)]
        Y = Y[rng.choice(len(Y), nY, replace=False)]
    z = np.vstack([X, Y])
    n, m = len(X), len(Y)
    stat_obs = stat_fn(X, Y)
    cnt = 0
    for _ in range(n_perm):
        idx = rng.permutation(len(z))
        Xp, Yp = z[idx[:n]], z[idx[n:]]
        s = stat_fn(Xp, Yp)
        if tail == 'right':
            cnt += (s >= stat_obs)
        elif tail == 'left':
            cnt += (s <= stat_obs)
        else:  # two-sided
            cnt += (abs(s) >= abs(stat_obs))
    pval = (cnt + 1) / (n_perm + 1)
    return stat_obs, pval

def mmd_stat(X, Y, gamma='median'):
    Z = np.vstack([X, Y])
    if gamma == 'median':
        # median heuristic
        idx = rng.choice(len(Z), size=min(2000, len(Z)), replace=False)
        Zs = Z[idx]
        d2 = np.sum((Zs[:,None,:]-Zs[None,:,:])**2, axis=2)
        med = np.median(d2[np.triu_indices_from(d2, 1)])
        gamma = 1.0 / (med + 1e-12)
    Kxx = rbf_kernel(X, X, gamma=gamma)
    Kyy = rbf_kernel(Y, Y, gamma=gamma)
    Kxy = rbf_kernel(X, Y, gamma=gamma)
    # u-statistic MMD^2
    np.fill_diagonal(Kxx, 0.0)
    np.fill_diagonal(Kyy, 0.0)
    m, n = len(X), len(Y)
    return Kxx.sum()/(m*(m-1)) + Kyy.sum()/(n*(n-1)) - 2*Kxy.mean()

def hotellings_T2_two_sample(X, Y):
    # 兩樣本 Hotelling’s T^2 （均值是否相等）、共變數用 Ledoit-Wolf 縮減，穩定高維
    n, m = len(X), len(Y)
    p = X.shape[1]
    mx, my = X.mean(axis=0), Y.mean(axis=0)
    # pooled covariance estimate via shrinkage
    Sx = LedoitWolf().fit(X - mx).covariance_
    Sy = LedoitWolf().fit(Y - my).covariance_
    Sp = ((n-1)*Sx + (m-1)*Sy) / (n+m-2)
    # T^2
    diff = (mx - my)
    # 數值穩定的逆
    from numpy.linalg import solve
    T2 = (n*m/(n+m)) * diff @ solve(Sp, diff)
    # 轉成 F 分佈
    # F = ((n+m-p-1)/(p*(n+m-2))) * T2 ~ F_{p, n+m-p-1}
    df1 = p
    df2 = n + m - p - 1
    if df2 <= 0:
        return T2, np.nan, (df1, df2)
    Fstat = ((n+m-p-1)/(p*(n+m-2))) * T2
    pval = 1 - f.cdf(Fstat, df1, df2)
    return T2, pval, (df1, df2)

def per_dim_tests_with_fdr(X, Y):
    pvals = np.array([ttest_ind(X[:,i], Y[:,i], equal_var=False).pvalue
                      for i in range(X.shape[1])])
    rej, pvals_fdr, _, _ = multipletests(pvals, method='fdr_bh')
    return pvals, pvals_fdr, rej

def quick_pca_plot(X, Y, title='PCA'):
    Z = np.vstack([X, Y])
    labels = np.array([0]*len(X) + [1]*len(Y))
    Z2 = PCA(n_components=2, random_state=0).fit_transform(Z)
    plt.figure(figsize=(5,4))
    plt.scatter(Z2[labels==0,0], Z2[labels==0,1], s=8, alpha=0.8, label='Group X')
    plt.scatter(Z2[labels==1,0], Z2[labels==1,1], s=8, alpha=0.8, label='Group Y')
    plt.title(title); plt.xlabel('PC1'); plt.ylabel('PC2'); plt.legend(); plt.show()


---

## Case A: 1000 維常態、各 5000 筆（總 1 萬）

In [ ]:
d = 1000
n1 = n2 = 5000

# 母體一樣（虛無）→ 可改成 shift[:30]=0.15 作為有差異的情境
mu1 = np.zeros(d)
mu2 = np.zeros(d)

# 也可做出相關性：這裡先用獨立常態
X = rng.standard_normal((n1, d)) + mu1
Y = rng.standard_normal((n2, d)) + mu2

# ---- 檢定們 ----
# 1) Energy distance（置換檢定，對高維穩健；可 subsample 加速）
estat, ep = permutation_test(energy_stat, X, Y, n_perm=300, subsample=4000)

# 2) MMD（RBF，置換檢定）
mmd, mmd_p = permutation_test(mmd_stat, X, Y, n_perm=300, subsample=4000)

# 3) Hotelling’s T^2（用 Ledoit-Wolf 縮減共變數）
T2, T2_p, dfs = hotellings_T2_two_sample(X, Y)

# 4) 逐維 + FDR
pvals, pvals_fdr, rej = per_dim_tests_with_fdr(X, Y)

print(f"[Energy] stat={estat:.4f}, p≈{ep:.4g}")
print(f"[MMD]    stat={mmd:.4f}, p≈{mmd_p:.4g}")
print(f"[T^2]    T2={T2:.2f}, p≈{T2_p}, df={dfs}")
print(f"[FDR]    significant dims = {rej.sum()} / {d}")

# 視覺化
neglogp = -np.log10(pvals + 1e-300)
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(pvals, bins=40, edgecolor='black'); plt.title('P-value histogram'); plt.xlabel('p'); plt.ylabel('count')
plt.subplot(1,2,2)
plt.scatter(range(d), neglogp, s=6, alpha=0.7, label='-log10 p')
if rej.any():
    plt.scatter(np.where(rej)[0], neglogp[rej], s=8, alpha=0.9, label='FDR sig')
plt.title('Per-dim -log10 p'); plt.legend(); plt.tight_layout(); plt.show()

quick_pca_plot(X, Y, title='PCA (Gaussian 1000D)')


理論上可以作但時間太久了

---

## Case B: MNIST：下載、分兩半，檢定「是否同母體」

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

# 下載 MNIST (784 維像素)
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
Xall = mnist.data.astype(np.float32)  # shape (70000, 784)
yall = mnist.target.astype(int)

# 常見前處理：標準化像素到 0-1 或標準化到零均值單位方差，二擇一
Xall = Xall / 255.0
# 或：Xall = StandardScaler().fit_transform(Xall)

# 分兩半（隨機洗牌後切）
idx = rng.permutation(len(Xall))
A = Xall[idx[:35000]]
B = Xall[idx[35000:]]

# ---- 檢定們 ----
# 可再次下採樣，否則置換檢定會很慢（這裡示範 subsample=6000）
estat, ep = permutation_test(energy_stat, A, B, n_perm=200, subsample=6000)
mmd, mmd_p = permutation_test(mmd_stat, A, B, n_perm=200, subsample=6000)
T2, T2_p, dfs = hotellings_T2_two_sample(A, B)
pvals, pvals_fdr, rej = per_dim_tests_with_fdr(A, B)

print(f"[MNIST Energy] stat={estat:.4f}, p≈{ep:.4g}")
print(f"[MNIST MMD]    stat={mmd:.4f}, p≈{mmd_p:.4g}")
print(f"[MNIST T^2]    T2={T2:.2e}, p≈{T2_p}, df={dfs}")
print(f"[MNIST FDR]    sig dims={rej.sum()} / {A.shape[1]}")

# 視覺化
neglogp = -np.log10(pvals + 1e-300)
plt.figure(figsize=(12,4))
plt.subplot(1,2,1); plt.hist(pvals, bins=40, edgecolor='black'); plt.title('MNIST per-pixel p histogram')
plt.subplot(1,2,2); plt.scatter(range(A.shape[1]), neglogp, s=5, alpha=0.7); plt.title('MNIST per-pixel -log10 p')
plt.tight_layout(); plt.show()

quick_pca_plot(A, B, title='PCA (MNIST halves)')
